# PostgreSQLを操作する

In [1]:
from dotenv import load_dotenv
from psycopg_pool import ConnectionPool
from sqlalchemy import create_engine, select, text
from sqlalchemy.ext.asyncio import create_async_engine, AsyncSession
from sqlalchemy.orm import DeclarativeBase, Mapped, mapped_column, Session
from sqlalchemy.types import Text
import os

In [2]:
load_dotenv()

conninfo = os.environ["POSTGRES_CONNINFO"]
url = os.environ["SQLARCHEMY_URL"]

# SQLログを見たい場合はTrueにする
echo = False

In [3]:
with ConnectionPool(conninfo=conninfo) as pool:
    with pool.connection() as session:
        with session.execute("select * from messages order by id asc") as cur:
            for record in cur:
                print(record)

(1, 'foo')
(2, 'bar')
(3, 'baz')
(4, 'qux')


## SQLAlchemy

In [4]:
engine = create_engine(url, echo=echo, echo_pool=echo)
try:
    # engine.connect()でもデータベースの操作はできるけれど、ORM的な機能を使うにはSessionを使うみたい。
    # この例では単純にクエリーを投げているだけなのでConnectionを使うのと変わらない使い方になってる。
    #with engine.connect() as conn:
    with Session(engine) as session:
        result = session.execute(text("select * from messages order by id asc"))
        print(result.all())

        result = session.execute(text("select * from messages where id = :id"), { "id": "1" })
        # 動的にカラム名が付けられたタプルがイテレートされる
        print([row.content for row in result.all()])

        session.execute(
            text("insert into messages (content) values (:content)"), 
            [{ "content": "hoge" }, { "content": "fuga" }],
        )
        #conn.commit()
        result = session.execute(text("select content from messages order by id asc"))
        # content, でタプルを分解して受け取っている
        print([content for content, in result.all()])
finally:
    engine.dispose()

[(1, 'foo'), (2, 'bar'), (3, 'baz'), (4, 'qux')]
['foo']
['foo', 'bar', 'baz', 'qux', 'hoge', 'fuga']


In [5]:
class Base(DeclarativeBase):
    pass

class Message(Base):
    __tablename__ = "messages"

    id: Mapped[int] = mapped_column(primary_key=True)
    content: Mapped[str] = mapped_column(Text())

    def __repr__(self):
        return f"Message(id={self.id}, content={self.content})"

engine = create_engine(url, echo=echo, echo_pool=echo)
try:
    with Session(engine) as session:
        session.add(Message(content="foobar"))

        m = session.execute(select(Message).filter_by(content="foobar")).scalar_one()
        print(m)
finally:
    engine.dispose()

Message(id=35, content=foobar)


### 非同期

In [6]:
async_engine = create_async_engine(url, echo=echo, echo_pool=echo)
try:

    session = AsyncSession(async_engine)
    try:
        result = await session.execute(select(Message).order_by(Message.id))
        for m in result.scalars():
            print(m)

    finally:
        await session.aclose()

finally:
    await async_engine.dispose()

Message(id=1, content=foo)
Message(id=2, content=bar)
Message(id=3, content=baz)
Message(id=4, content=qux)
